In [31]:
import os
import berserk
import pandas as pd

from tqdm.notebook import tqdm

In [32]:
LICHESS_API_TOKEN = os.environ["LICHESS_API_TOKEN"]
# LICHESS_USERNAME = "rootsec1"
LICHESS_USERNAME = "ruchitoshniwal1"

In [33]:
"""Lichess API client."""

session = berserk.TokenSession(LICHESS_API_TOKEN)
client = berserk.Client(session=session)


def get_games_and_moves_by_username(username: str) -> list[dict]:
    """Get the games and moves of a user by their username."""
    games = client.games.export_by_player(username, analysed=False, evals=False, moves=True)
    game_list = []
    for game in tqdm(games):
        game_id = game.get("id", "")
        move_list = game.get("moves", "")
        variant = game.get("speed", "")
        end_reason = game.get("status", "")

        players = game.get("players", {})
        white_player = players.get("white", {}).get("user", {}).get("name", "")
        black_player = players.get("black", {}).get("user", {}).get("name", "")
        winning_player = game.get("winner", "")
        winning_player = white_player if winning_player == "white" else black_player

        game_summary_dict = {
            "game_id": game_id,
            "variant": variant,
            "end_reason": end_reason,
            "white_player": white_player,
            "black_player": black_player,
            "winning_player": winning_player,
            "move_list": move_list,
        }
        game_list.append(game_summary_dict)
    return game_list


In [34]:
game_list = get_games_and_moves_by_username(LICHESS_USERNAME)
df = pd.DataFrame(game_list)
df.head()

0it [00:00, ?it/s]

,game_id,variant,end_reason,white_player,black_player,winning_player,move_list
0,IxgcuAMI,bullet,mate,ruchitoshniwal1,Username54321,Username54321,d4 d6 Nf3 Nf6 Bg5 c6 c4 Qa5+ Nc3 Bg4 e3 Bxf3 Q...
1,U6QJVSi6,bullet,mate,piotrex,ruchitoshniwal1,ruchitoshniwal1,d4 d5 Nf3 Nf6 c4 c5 dxc5 e6 cxd5 exd5 Nc3 Bxc5...
2,ErC4K1Ps,correspondence,resign,,ruchitoshniwal1,,e4 e5 Nf3 Nc6 Nc3 Nf6 Bb5 Bc5 Bxc6 dxc6 d4 exd...
3,waSRX000,correspondence,mate,,ruchitoshniwal1,ruchitoshniwal1,e4 e5 d4 Nc6 Nf3 Nf6 Nc3 exd4 Nxd4 Bc5 Nxc6 bx...
4,MX85BjR4,correspondence,mate,ruchitoshniwal1,,ruchitoshniwal1,e4 e5 Nf3 Nc6 Bc4 Nf6 Nc3 Bb4 a3 Bxc3 dxc3 Nxe...


In [35]:
# Fill "white_player", "black_player" and "winning_player" columns containing an empty string with "Anonymous"
df = df.dropna()
df["white_player"] = df["white_player"].replace("", "ANONYMOUS")
df["black_player"] = df["black_player"].replace("", "ANONYMOUS")
df["winning_player"] = df["winning_player"].replace("", "ANONYMOUS")
df = df.set_index("game_id")
df

,variant,end_reason,white_player,black_player,winning_player,move_list
game_id,,,,,,
IxgcuAMI,bullet,mate,ruchitoshniwal1,Username54321,Username54321,d4 d6 Nf3 Nf6 Bg5 c6 c4 Qa5+ Nc3 Bg4 e3 Bxf3 Q...
U6QJVSi6,bullet,mate,piotrex,ruchitoshniwal1,ruchitoshniwal1,d4 d5 Nf3 Nf6 c4 c5 dxc5 e6 cxd5 exd5 Nc3 Bxc5...
ErC4K1Ps,correspondence,resign,ANONYMOUS,ruchitoshniwal1,ANONYMOUS,e4 e5 Nf3 Nc6 Nc3 Nf6 Bb5 Bc5 Bxc6 dxc6 d4 exd...
waSRX000,correspondence,mate,ANONYMOUS,ruchitoshniwal1,ruchitoshniwal1,e4 e5 d4 Nc6 Nf3 Nf6 Nc3 exd4 Nxd4 Bc5 Nxc6 bx...
MX85BjR4,correspondence,mate,ruchitoshniwal1,ANONYMOUS,ruchitoshniwal1,e4 e5 Nf3 Nc6 Bc4 Nf6 Nc3 Bb4 a3 Bxc3 dxc3 Nxe...
...,...,...,...,...,...,...
UQyLLJmb,correspondence,draw,ruchitoshniwal1,ANONYMOUS,ANONYMOUS,e4 d5 exd5 Qxd5 Nf3 Qe4+ Be2 Qe5 O-O Qd6 Nc3 Q...
pTsypag5,correspondence,mate,ruchitoshniwal1,ANONYMOUS,ruchitoshniwal1,e4 e5 Nf3 Bc5 Bc4 Bb4 Nxe5 Qe7 Qh5 Qc5 O-O Qxc...
udCetnJ4,correspondence,mate,ruchitoshniwal1,ANONYMOUS,ANONYMOUS,e4 e5 Bc4 Bc5 Nc3 Qh4 Nf3 Qxf2#


In [36]:
df.to_csv(f"games_{LICHESS_USERNAME}.csv")